In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X = df.iloc[:, :-1].values
y=df.iloc[:,-1].values

In [5]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [6]:
X.shape

(768, 8)

In [7]:
y.shape

(768,)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
X = scaler.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [12]:
import tensorflow as tf


In [13]:
LAYERS = [
    tf.keras.layers.Dense(32, activation='relu', input_dim=8),
    tf.keras.layers.Dense(1, activation='sigmoid')
]

model = tf.keras.models.Sequential(LAYERS)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [15]:
LOSS_FUNCTION = 'binary_crossentropy'
OPTIMIZER = 'Adam'
METRICS = ['accuracy']

model.compile(loss=LOSS_FUNCTION,
              optimizer = OPTIMIZER,
              metrics= METRICS)

In [16]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 6s 13ms/step - loss: 0.7099 - accuracy: 0.5342 - val_loss: 0.6899 - val_accuracy: 0.5714
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.6457 - accuracy: 0.6303 - val_loss: 0.6415 - val_accuracy: 0.6429
Epoch 3/10
20/20 [==============================] - 0s 8ms/step - loss: 0.6005 - accuracy: 0.6808 - val_loss: 0.6090 - val_accuracy: 0.6753
Epoch 4/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5669 - accuracy: 0.7199 - val_loss: 0.5841 - val_accuracy: 0.7078
Epoch 5/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5439 - accuracy: 0.7264 - val_loss: 0.5669 - val_accuracy: 0.7208
Epoch 6/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5253 - accuracy: 0.7508 - val_loss: 0.5525 - val_accuracy: 0.7532
Epoch 7/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5107 - accuracy: 0.7541 - val_loss: 0.5425 - val_accuracy: 0.7532
Epoch 8/10
20/20 [=

In [17]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.9 MB/s eta 0:00:00


In [19]:
import keras_tuner as kt

Hyper parameter...

In [32]:
# from keras.optimizers import rmsprop
def build_model(hp):
  LAYERS = [
    tf.keras.layers.Dense(32, activation='relu', input_dim=8),
    tf.keras.layers.Dense(1, activation='sigmoid')
]

  model = tf.keras.models.Sequential(LAYERS)

  LOSS_FUNCTION = 'binary_crossentropy'
  OPTIMIZER = hp.Choice('optimizer', values = ['adam','sgd', 'rmsprop','adadelata'])
  METRICS = ['accuracy']

  model.compile(loss=LOSS_FUNCTION,
                optimizer = OPTIMIZER,
                metrics= METRICS)
  return model


In [33]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [34]:
tuner = kt.RandomSearch(build_model, objective ='val_accuracy', max_trials=5)

In [37]:
tuner.get_best_hyperparameters()[0].values # best optimizer

{'optimizer': 'adam'}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(X_train, y_train, epochs=10, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/10
20/20 [==============================] - 2s 12ms/step - loss: 0.5335 - accuracy: 0.7508 - val_loss: 0.5340 - val_accuracy: 0.7273
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5182 - accuracy: 0.7557 - val_loss: 0.5244 - val_accuracy: 0.7532
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5059 - accuracy: 0.7541 - val_loss: 0.5155 - val_accuracy: 0.7597
Epoch 10/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4955 - accuracy: 0.7557 - val_loss: 0.5075 - val_accuracy: 0.7597


how to select number of node/neuron in models

In [42]:
# from keras.optimizers import rmsprop
def build_model(hp):

  units = hp.Int('units',min_value=8, max_value = 128, step=8)

  LAYERS = [
    tf.keras.layers.Dense(units = units, activation='relu', input_dim=8),
    tf.keras.layers.Dense(1, activation='sigmoid')
]

  model = tf.keras.models.Sequential(LAYERS)

  LOSS_FUNCTION = 'binary_crossentropy'
  OPTIMIZER = 'rmsprop'
  METRICS = ['accuracy']

  model.compile(loss=LOSS_FUNCTION,
                optimizer = OPTIMIZER,
                metrics= METRICS)
  return model


In [46]:
tuner = kt.RandomSearch(build_model, objective ='val_accuracy', max_trials=5, directory='mydir')

In [47]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 12s


In [48]:
tuner.get_best_hyperparameters()[0].values # this is best parameters...

{'units': 104}

In [50]:
model = tuner.get_best_models(num_models=1)[0]

In [51]:
model

how to select number of layers

In [53]:
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential

In [54]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(104, activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
      model.add(Dense(104, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  LOSS_FUNCTION = 'binary_crossentropy'
  OPTIMIZER = 'rmsprop'
  METRICS = ['accuracy']

  model.compile(loss=LOSS_FUNCTION,
                optimizer = OPTIMIZER,
                metrics= METRICS)
  return model

In [55]:
tuner = kt.RandomSearch(build_model, objective ='val_accuracy', max_trials=5, directory='dir1')

In [56]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 21s


In [57]:
tuner.get_best_hyperparameters()[0].values # this is best parameters...

{'num_layers': 2}

In [58]:
model = tuner.get_best_models(num_models=1)[0]

In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 104)               10920     
                                                                 
 dense_2 (Dense)             (None, 104)               10920     
                                                                 
 dense_3 (Dense)             (None, 1)                 105       
                                                                 
Total params: 22,881
Trainable params: 22,881
Non-trainable params: 0
_________________________________________________________________


ALL in one model...

In [60]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value= 1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh']),
                      input_dim=8))


    else:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh'])))

    counter += 1


  model.add(Dense(1, activation='sigmoid'))


  LOSS_FUNCTION = "binary_crossentropy"
  OPTIMIZER = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  METRICS = ["accuracy"]

  model.compile(loss=LOSS_FUNCTION,
                optimizer=OPTIMIZER,
                metrics=METRICS)

  return model


In [61]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='mydir2')

In [62]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 26s


In [66]:
model = tuner.get_best_models(num_models=1)[0]

In [67]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                720       
                                                                 
 dense_1 (Dense)             (None, 16)                1296      
                                                                 
 dense_2 (Dense)             (None, 96)                1632      
                                                                 
 dense_3 (Dense)             (None, 72)                6984      
                                                                 
 dense_4 (Dense)             (None, 120)               8760      
                                                                 
 dense_5 (Dense)             (None, 80)                9680      
                                                                 
 dense_6 (Dense)             (None, 8)                 6

In [68]:
model.fit(X_train, y_train, epochs=10, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/10
20/20 [==============================] - 4s 25ms/step - loss: 0.5705 - accuracy: 0.7801 - val_loss: 0.5175 - val_accuracy: 0.7857
Epoch 8/10
20/20 [==============================] - 0s 10ms/step - loss: 0.4746 - accuracy: 0.7801 - val_loss: 0.5221 - val_accuracy: 0.7468
Epoch 9/10
20/20 [==============================] - 0s 9ms/step - loss: 0.4533 - accuracy: 0.7850 - val_loss: 0.5799 - val_accuracy: 0.7532
Epoch 10/10
20/20 [==============================] - 0s 11ms/step - loss: 0.4454 - accuracy: 0.7915 - val_loss: 0.5215 - val_accuracy: 0.7727
